# Requirements

In [1]:
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from os import path, makedirs, listdir
d_targets = '/content/drive/MyDrive/Modeling Satelities Images Building Damaged/data/train'
print(d_targets,'\n','Quantidade de máscaras: ',len(listdir(path.join(d_targets, 'targets'))))

/content/drive/MyDrive/Modeling Satelities Images Building Damaged/data/train 
 Quantidade de máscaras:  5598


# Functions

In [ ]:
import os
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import numpy as np
np.random.seed(1)
import random
random.seed(1)
import pandas as pd
import cv2
import timeit
from os import path, makedirs, listdir
import sys
sys.setrecursionlimit(10000)
from multiprocessing import Pool
#from skimage.morphology import square, dilation, watershed, erosion
#from skimage import io

from shapely.wkt import loads
from shapely.geometry import mapping, Polygon

# import matplotlib.pyplot as plt
# import seaborn as sns

import json

masks_dir = 'targets'

train_dirs = ['/content/drive/MyDrive/Modeling Satelities Images Building Damaged/data/tier3']


def mask_for_polygon(poly, im_size=(1024, 1024)):
    img_mask = np.zeros(im_size, np.uint8)
    int_coords = lambda x: np.array(x).round().astype(np.int32)
    exteriors = [int_coords(poly.exterior.coords)]
    interiors = [int_coords(pi.coords) for pi in poly.interiors]
    cv2.fillPoly(img_mask, exteriors, 1)
    cv2.fillPoly(img_mask, interiors, 0)
    return img_mask


damage_dict = {
    "no-damage": 1,
    "minor-damage": 2,
    "major-damage": 3,
    "destroyed": 4,
    "un-classified": 1 # ?
}


def process_image(json_file):
    js1 = json.load(open(json_file))
    js2 = json.load(open(json_file.replace('_pre_disaster', '_post_disaster')))

    msk = np.zeros((1024, 1024), dtype='uint8')
    msk_damage = np.zeros((1024, 1024), dtype='uint8')

    for feat in js1['features']['xy']:
        poly = loads(feat['wkt'])
        _msk = mask_for_polygon(poly)
        msk[_msk > 0] = 255

    for feat in js2['features']['xy']:
        poly = loads(feat['wkt'])
        subtype = feat['properties']['subtype']
        _msk = mask_for_polygon(poly)
        msk_damage[_msk > 0] = damage_dict[subtype]

    cv2.imwrite(json_file.replace('/labels/', '/targets/').replace('_pre_disaster.json', '_pre_disaster.png').replace('.png', '_target.png'), msk, [cv2.IMWRITE_PNG_COMPRESSION, 9])
    cv2.imwrite(json_file.replace('/labels/', '/targets/').replace('_pre_disaster.json', '_post_disaster.png').replace('.png', '_target.png'), msk_damage, [cv2.IMWRITE_PNG_COMPRESSION, 9])

In [ ]:
t0 = timeit.default_timer()

all_files = []
for d in train_dirs:
    makedirs(path.join(d, masks_dir), exist_ok=True)
    print(d,'\n','Quantidade de imagens: ',len(listdir(path.join(d, 'images'))))
    for f in sorted(listdir(path.join(d, 'images'))):
        if '_pre_disaster.png' in f:
            all_files.append(path.join(d, 'labels', f.replace('_pre_disaster.png', '_pre_disaster.json')))


with Pool() as pool:
    _ = pool.map(process_image, all_files)

elapsed = timeit.default_timer() - t0
print('Time: {:.3f} min'.format(elapsed / 60))

d_targets = '/content/drive/MyDrive/Modeling Satelities Images Building Damaged/data/tier3'
print(d_targets,'\n','Quantidade de máscaras: ',len(listdir(path.join(d_targets, 'targets'))))

/content/drive/MyDrive/Modeling Satelities Images Building Damaged/data/tier3 
 Quantidade de imagens:  12726
Time: 16.574 min
/content/drive/MyDrive/Modeling Satelities Images Building Damaged/data/tier3 
 Quantidade de máscaras:  12726


In [ ]:
import os
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import numpy as np
np.random.seed(1)
import random
random.seed(1)
import pandas as pd
import cv2
import timeit
from os import path, makedirs, listdir
import sys
sys.setrecursionlimit(10000)
from multiprocessing import Pool
#from skimage.morphology import square, dilation, watershed, erosion
#from skimage import io

from shapely.wkt import loads
from shapely.geometry import mapping, Polygon

# import matplotlib.pyplot as plt
# import seaborn as sns

import json

masks_dir = 'targets2'

train_dirs = ['/content/drive/MyDrive/Modeling Satelities Images Building Damaged/data/hold']


def mask_for_polygon(poly, im_size=(1024, 1024)):
    img_mask = np.zeros(im_size, np.uint8)
    int_coords = lambda x: np.array(x).round().astype(np.int32)
    exteriors = [int_coords(poly.exterior.coords)]
    interiors = [int_coords(pi.coords) for pi in poly.interiors]
    cv2.fillPoly(img_mask, exteriors, 1)
    cv2.fillPoly(img_mask, interiors, 0)
    return img_mask


damage_dict = {
    "no-damage": 1,
    "minor-damage": 2,
    "major-damage": 3,
    "destroyed": 4,
    "un-classified": 1 # ?
}


def process_image(json_file):
    js1 = json.load(open(json_file))
    js2 = json.load(open(json_file.replace('_pre_disaster', '_post_disaster')))

    msk = np.zeros((1024, 1024), dtype='uint8')
    msk_damage = np.zeros((1024, 1024), dtype='uint8')

    for feat in js1['features']['xy']:
        poly = loads(feat['wkt'])
        _msk = mask_for_polygon(poly)
        msk[_msk > 0] = 255

    for feat in js2['features']['xy']:
        poly = loads(feat['wkt'])
        subtype = feat['properties']['subtype']
        _msk = mask_for_polygon(poly)
        msk_damage[_msk > 0] = damage_dict[subtype]

    cv2.imwrite(json_file.replace('/labels/', '/targets2/').replace('_pre_disaster.json', '_pre_disaster.png').replace('.png', '_target.png'), msk, [cv2.IMWRITE_PNG_COMPRESSION, 9])
    cv2.imwrite(json_file.replace('/labels/', '/targets2/').replace('_pre_disaster.json', '_post_disaster.png').replace('.png', '_target.png'), msk_damage, [cv2.IMWRITE_PNG_COMPRESSION, 9])

In [ ]:
t0 = timeit.default_timer()

all_files = []
for d in train_dirs:
    makedirs(path.join(d, masks_dir), exist_ok=True)
    print(d,'\n','Quantidade de imagens: ',len(listdir(path.join(d, 'images'))))
    for f in sorted(listdir(path.join(d, 'images'))):
        if '_pre_disaster.png' in f:
            all_files.append(path.join(d, 'labels', f.replace('_pre_disaster.png', '_pre_disaster.json')))


with Pool() as pool:
    _ = pool.map(process_image, all_files)

elapsed = timeit.default_timer() - t0
print('Time: {:.3f} min'.format(elapsed / 60))

d_targets = '/content/drive/MyDrive/Modeling Satelities Images Building Damaged/data/hold'
print(d_targets,'\n','Quantidade de máscaras: ',len(listdir(path.join(d_targets, 'targets2'))))

/content/drive/MyDrive/Modeling Satelities Images Building Damaged/data/hold 
 Quantidade de imagens:  1866
Time: 3.241 min
/content/drive/MyDrive/Modeling Satelities Images Building Damaged/data/hold 
 Quantidade de máscaras:  1866


In [ ]:
import os
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import numpy as np
np.random.seed(1)
import random
random.seed(1)
import pandas as pd
import cv2
import timeit
from os import path, makedirs, listdir
import sys
sys.setrecursionlimit(10000)
from multiprocessing import Pool
#from skimage.morphology import square, dilation, watershed, erosion
#from skimage import io

from shapely.wkt import loads
from shapely.geometry import mapping, Polygon

# import matplotlib.pyplot as plt
# import seaborn as sns

import json

masks_dir = 'targets'

train_dirs = ['/content/drive/MyDrive/Modeling Satelities Images Building Damaged/data/test']


def mask_for_polygon(poly, im_size=(1024, 1024)):
    img_mask = np.zeros(im_size, np.uint8)
    int_coords = lambda x: np.array(x).round().astype(np.int32)
    exteriors = [int_coords(poly.exterior.coords)]
    interiors = [int_coords(pi.coords) for pi in poly.interiors]
    cv2.fillPoly(img_mask, exteriors, 1)
    cv2.fillPoly(img_mask, interiors, 0)
    return img_mask


damage_dict = {
    "no-damage": 1,
    "minor-damage": 2,
    "major-damage": 3,
    "destroyed": 4,
    "un-classified": 1 # ?
}


def process_image(json_file):
    js1 = json.load(open(json_file))
    js2 = json.load(open(json_file.replace('_pre_disaster', '_post_disaster')))

    msk = np.zeros((1024, 1024), dtype='uint8')
    msk_damage = np.zeros((1024, 1024), dtype='uint8')

    for feat in js1['features']['xy']:
        poly = loads(feat['wkt'])
        _msk = mask_for_polygon(poly)
        msk[_msk > 0] = 255

    for feat in js2['features']['xy']:
        poly = loads(feat['wkt'])
        subtype = feat['properties']['subtype']
        _msk = mask_for_polygon(poly)
        msk_damage[_msk > 0] = damage_dict[subtype]

    cv2.imwrite(json_file.replace('/labels/', '/targets/').replace('_pre_disaster.json', '_pre_disaster.png').replace('.png', '_target.png'), msk, [cv2.IMWRITE_PNG_COMPRESSION, 9])
    cv2.imwrite(json_file.replace('/labels/', '/targets/').replace('_pre_disaster.json', '_post_disaster.png').replace('.png', '_target.png'), msk_damage, [cv2.IMWRITE_PNG_COMPRESSION, 9])

In [ ]:
t0 = timeit.default_timer()

all_files = []
for d in train_dirs:
    makedirs(path.join(d, masks_dir), exist_ok=True)
    print(d,'\n','Quantidade de imagens: ',len(listdir(path.join(d, 'images'))))
    for f in sorted(listdir(path.join(d, 'images'))):
        if '_pre_disaster.png' in f:
            all_files.append(path.join(d, 'labels', f.replace('_pre_disaster.png', '_pre_disaster.json')))


with Pool() as pool:
    _ = pool.map(process_image, all_files)

elapsed = timeit.default_timer() - t0
print('Time: {:.3f} min'.format(elapsed / 60))

d_targets = '/content/drive/MyDrive/Modeling Satelities Images Building Damaged/data/test'
print(d_targets,'\n','Quantidade de máscaras: ',len(listdir(path.join(d_targets, 'targets'))))

/content/drive/MyDrive/Modeling Satelities Images Building Damaged/data/test 
 Quantidade de imagens:  1839
Time: 1.893 min
/content/drive/MyDrive/Modeling Satelities Images Building Damaged/data/test 
 Quantidade de máscaras:  1838


In [ ]:
import os
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import numpy as np
np.random.seed(1)
import random
random.seed(1)
import pandas as pd
import cv2
import timeit
from os import path, makedirs, listdir
import sys
sys.setrecursionlimit(10000)
from multiprocessing import Pool
#from skimage.morphology import square, dilation, watershed, erosion
#from skimage import io

from shapely.wkt import loads
from shapely.geometry import mapping, Polygon

# import matplotlib.pyplot as plt
# import seaborn as sns

import json

masks_dir = 'targets'

train_dirs = ['/content/drive/MyDrive/Modeling Satelities Images Building Damaged/data/train']


def mask_for_polygon(poly, im_size=(1024, 1024)):
    img_mask = np.zeros(im_size, np.uint8)
    int_coords = lambda x: np.array(x).round().astype(np.int32)
    exteriors = [int_coords(poly.exterior.coords)]
    interiors = [int_coords(pi.coords) for pi in poly.interiors]
    cv2.fillPoly(img_mask, exteriors, 1)
    cv2.fillPoly(img_mask, interiors, 0)
    return img_mask


damage_dict = {
    "no-damage": 1,
    "minor-damage": 2,
    "major-damage": 3,
    "destroyed": 4,
    "un-classified": 1 # ?
}


def process_image(json_file):
    js1 = json.load(open(json_file))
    js2 = json.load(open(json_file.replace('_pre_disaster', '_post_disaster')))

    msk = np.zeros((1024, 1024), dtype='uint8')
    msk_damage = np.zeros((1024, 1024), dtype='uint8')

    for feat in js1['features']['xy']:
        poly = loads(feat['wkt'])
        _msk = mask_for_polygon(poly)
        msk[_msk > 0] = 255

    for feat in js2['features']['xy']:
        poly = loads(feat['wkt'])
        subtype = feat['properties']['subtype']
        _msk = mask_for_polygon(poly)
        msk_damage[_msk > 0] = damage_dict[subtype]

    cv2.imwrite(json_file.replace('/labels/', '/targets/').replace('_pre_disaster.json', '_pre_disaster.png').replace('.png', '_target.png'), msk, [cv2.IMWRITE_PNG_COMPRESSION, 9])
    cv2.imwrite(json_file.replace('/labels/', '/targets/').replace('_pre_disaster.json', '_post_disaster.png').replace('.png', '_target.png'), msk_damage, [cv2.IMWRITE_PNG_COMPRESSION, 9])

In [ ]:
t0 = timeit.default_timer()

all_files = []
for d in train_dirs:
    makedirs(path.join(d, masks_dir), exist_ok=True)
    print(d,'\n','Quantidade de imagens: ',len(listdir(path.join(d, 'images'))))
    for f in sorted(listdir(path.join(d, 'images'))):
        if '_pre_disaster.png' in f:
            all_files.append(path.join(d, 'labels', f.replace('_pre_disaster.png', '_pre_disaster.json')))


with Pool() as pool:
    _ = pool.map(process_image, all_files)

elapsed = timeit.default_timer() - t0
print('Time: {:.3f} min'.format(elapsed / 60))

d_targets = '/content/drive/MyDrive/Modeling Satelities Images Building Damaged/data/train'
print(d_targets,'\n','Quantidade de máscaras: ',len(listdir(path.join(d_targets, 'targets'))))

/content/drive/MyDrive/Modeling Satelities Images Building Damaged/data/train 
 Quantidade de imagens:  5598
Time: 5.130 min
/content/drive/MyDrive/Modeling Satelities Images Building Damaged/data/train 
 Quantidade de máscaras:  5598
